# 호텔 리뷰 데이터 : 감성 분류& 긍정/부정 키워드 분석

##개요


제주 호텔의 리뷰 데이터(평가 점수 + 평가 내용)을 활용해 다음 2가지 분석을 진행합니다:

리뷰속에 담긴 사람의 긍정 / 부정 감성을 파악하여 분류할 수 있는 감성 분류 예측 모델을 만든다

만든 모델을 활용해 긍정 / 부정 키워드를 출력해, 이용객들이 느낀 제주 호텔의 장,단점을 파악한다


1. Library & Data import

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

사용할 데이터셋

Tripadvisor 여행사이트에서 "제주 호텔"로 검색해서 나온 리뷰들을 활용합니다. (평점 & 평가 내용 포함)


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/tripadviser_review.csv")

In [ ]:
df.head()

### Feature Description

rating: 이용자 리뷰의 평가 점수 (1~5)

text: 이용자 리뷰 평가 내용


In [ ]:
df.shape

In [ ]:
#missing value
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df['text'][0]

In [ ]:
df['text'][100]

내용을 확인해보니 소량의 특수문자와 모음이 존재하는 경우가 있으니 정규표현식으로 제거해보자

In [ ]:
# 정규 표현식 함수 정의

import re

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [ ]:
df['text'][0]

In [ ]:
apply_regular_expression(df['text'][0])

### 한국어 형태소 분석- 명사 단위

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
from collections import Counter

In [ ]:
apply_regular_expression(df['text'][0])

In [ ]:
okt = Okt() #명사 형태소 추출 함수
nouns = okt.nouns(apply_regular_expression(df['text'][0]))
nouns

전체 말뭉치(Corpus)에 적용해서 명사 형태소를 추출

In [ ]:
#말물치 생성
corpus = "".join(df['text'].tolist())
# tolist() 함수를 사용하여 같은 레벨(위치)에 있는 데이터 끼리 묶어준다
corpus

In [ ]:
#정규표현식 적용
apply_regular_expression(corpus)

In [ ]:
#전체 말뭉치(Corpus)에서 명하 형태소 추출
nouns = okt.nouns(apply_regular_expression(corpus))
nouns

In [ ]:
#빈도 탐색
counter = Counter(nouns)

In [ ]:
counter.most_common(10)

#### 한글자 명사 제거
위 결과에서 보이듯이, 두 글자 키워드가 대부분 의미 있는 단어지만, ‘수’, ‘것’, '곳’과 같은 한 글자 키워드는 분석에 딱히 좋은 영향을 미치지 않은 것으로 보입니다.

In [ ]:
available_counter = Counter({x: counter[x] for x in counter if len(x) > 1})
available_counter.most_common(10)

이제 한글자 키워드 모두 제거됐습니다. 하지만 “우리”, “매우” 와 같은 실질적인 의미가 없고 꾸민 역할을 하는 불용어들 아직 존재합니다. 한국어 불용어 사전을 정의하여 불용어도 제거해줄게요.

[RANKS NL](https://www.ranks.nl/)에 제공해주는 [한국어 불용어 사전](https://www.ranks.nl/stopwords/korean)을 활용하겠습니다.

In [ ]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

이 외에도 우리가 분석하고자 하는 데이터셋에 특화된 불용어들이 있습니다. 예를 들면: “제주”, “호텔”, “숙소” 등. 이런 단어들도 불용어 사전에 추가해보도록 할게요.

In [ ]:
jeju_hotel_stopwords = ['제주', '제주도', '호텔', '리뷰', '숙소', '여행', '트립']
for word in jeju_hotel_stopwords:
    stopwords.append(word)

## word Count

> BOW 벡터 생성

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(df['text'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [ ]:
#단어 리스트
word_list

In [ ]:
#각 단어가 전체 리뷰중에 등장한 총 횟수
count_list

In [ ]:
#각 단어의 리뷰별 등장 횟수
bow_vect.toarray()

In [ ]:
bow_vect.shape

In [ ]:
# "단어" - "총 등장 횟수" Matching

word_count_dict = dict(zip(word_list, count_list))
word_count_dict

## TF-IDF 적용

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)


In [ ]:
print(tf_idf_vect.shape)

변환 후 1001*3599 matrix가 출력됩니다. 여기서

한 행(row)은 한 리뷰를 의미하고

한 열(column)은 한 단어를 의미합니다.


In [ ]:
# 첫 번째 리뷰에서의 단어 중요도(TF-IDF 값) -- 0이 아닌 것만 출력
print(tf_idf_vect[0])

In [ ]:
# 첫 번째 리뷰에서 모든 단어의 중요도 -- 0인 값까지 포함
print(tf_idf_vect[0].toarray().shape)
print(tf_idf_vect[0].toarray())

- "벡터"-"단어" mapping

In [ ]:
vect.vocabulary_

In [ ]:
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'...')

## 감성 분류 – Logistic Regression

이제 전처리된 리뷰 데이터를 활용하여 감성 분류 예측 모델을 만들겠습니다.

감성 분류 예측 모델이란, 이용자 리뷰의 평가 내용을 통해 이 리뷰가 긍정적인지, 부정적인지를 예측하여, 이용자의 감성을 파악하는 겁니다.

따라서, 모델의 X 값(즉, feature 값)은 이용자 리뷰의 평가 내용이 되겠고, 모델의 Y 값(즉, label 값)은 이용자의 긍/부정 감성이 되겠습니다.

데이터셋 생성

Label

우리는 이용자의 리뷰를 “긍정” / “부정” 두가지 부류로 나누고자 합니다. 하지만 이러한 이용자의 감성을 대표할 수 있는 “평가 점수” 변수는 1 ~ 5의 value를 가지고 있습니다. 따라서 "평가 점수"변수 (rating: 1 ~ 5)를 이진 변수 (긍정: 1, 부정:0)으로 변환해야 합니다.


In [ ]:
df.sample(10)

리뷰 내용와 평점을 살펴보면, 4 ~ 5점 리뷰는 대부분 긍정적이었지만, 1 ~ 3점 리뷰에서는 부정적인 평가가 좀 많이 보였습니다.
그래서 4점, 5점인 리뷰는 "긍정적인 리뷰"로 분류하여 1를 부여하고, 1 ~ 3점 리뷰는 "부정적인 리뷰"로 분류하여 0을 부여하도록 할게요.

In [ ]:
df['rating'].hist()

In [ ]:
def rating_to_label(rating):
    if rating > 3:
        return 1
    else:
        return 0
    
df['y'] = df['rating'].apply(lambda x: rating_to_label(x))

In [ ]:
df.head()

In [ ]:
df['y'].value_counts()

- Train,test Split

In [ ]:
from sklearn.model_selection import train_test_split

x = tf_idf_vect
y = df['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# fit in training set
lr = LogisticRegression(random_state = 0)
lr.fit(x_train, y_train)

# predict in test set
y_pred = lr.predict(x_test)

## 분류 결과 평가

In [ ]:
# classification result for test set

print('accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('precision: %.2f' % precision_score(y_test, y_pred))
print('recall: %.2f' % recall_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))

In [ ]:
# confusion matrix

from sklearn.metrics import confusion_matrix

confu = confusion_matrix(y_true = y_test, y_pred = y_pred)

plt.figure(figsize=(4, 3))
sns.heatmap(confu, annot=True, annot_kws={'size':15}, cmap='OrRd', fmt='.10g')
plt.title('Confusion Matrix')
plt.show()

모델 평가결과를 살펴보면, 모델이 지나치게 긍정(“1”)으로만 예측하는 경향이 있습니다. 따라서 긍정 리뷰를 잘 예측하지만, 부정 리뷰에 대한 예측 정확도가 매우 낮습니다. 이는 샘플데이터의 클래스 불균형으로 인한 문제로 보입니다.
따라서, 클래스 불균형 조정을 진행하겠습니다.


In [ ]:
df['y'].value_counts()

In [ ]:
positive_random_idx = df[df['y']==1].sample(275, random_state=12).index.tolist()
negative_random_idx = df[df['y']==0].sample(275, random_state=12).index.tolist()

In [ ]:
random_idx = positive_random_idx + negative_random_idx
x = tf_idf_vect[random_idx]
y = df['y'][random_idx]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

In [ ]:
x_train.shape,y_train.shape

In [ ]:
x_test.shape,y_test.shape

모델 재학습

In [ ]:
lr2 = LogisticRegression(random_state = 0)
lr2.fit(x_train, y_train)
y_pred = lr2.predict(x_test)

분류 결과 평가

In [ ]:
# classification result for test set

print('accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('precision: %.2f' % precision_score(y_test, y_pred))
print('recall: %.2f' % recall_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))

In [ ]:
# confusion matrix

from sklearn.metrics import confusion_matrix

confu = confusion_matrix(y_true = y_test, y_pred = y_pred)

plt.figure(figsize=(4, 3))
sns.heatmap(confu, annot=True, annot_kws={'size':15}, cmap='OrRd', fmt='.10g')
plt.title('Confusion Matrix')
plt.show()

#### 긍정 / 부정 키워드 분석


기계는 이처럼 리뷰 내용에 나타나는 사람의 감성을 구별할 수 있을 뿐만 아니라, 학습된 Logistic Regression 모델을 이용하여 긍/부정 키워드도 추출해낼 수 있습니다.

추출된 키워드를 통해서 이용자가 느끼는 제주호델의 장,단점을 파악할 수 있고, 이를 기반으로 앞으로 유지해야 할 좋은 서비스와 개선이 필요한 아쉬운 서비스에 대해서도 어느정도 판단할 수 있습니다.


긍 / 부정 키워드를 추출하기 위해 먼저 Logistic Regression 모델에 각 단어의 coeficient를 시각화해보겠습니다.

In [ ]:
lr2.coef_

In [ ]:
# print logistic regression's coef

plt.figure(figsize=(10, 8))
plt.bar(range(len(lr2.coef_[0])), lr2.coef_[0])

여기서 계수가 양인 경우는 단어가 긍정적인 영향을 미쳤다고 볼 수 있고, 반면에, 음인 경우는 부정적인 영향을 미쳤다고 볼 수 있습니다.

이 계수들을 크기순으로 정렬하면, 긍정 / 부정 키워드를 출력하는 지표가 되겠습니다.

In [ ]:
#긍정 키워드와 부정 키워드의 Top5를 각각 출력
print(sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = True)[:5])
print(sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = True)[-5:])
# enumerate: 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환함

이처럼 단어의 coeficient와 index가 출력이 됩니다.


이제 전체 단어가 포함한 "긍정 키워드 리스트"와 "부정 키워드 리스트"를 정의하고 출력해볼게요.

In [ ]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = True)
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = False)
coef_pos_index

In [ ]:
#index를 변환하여 '긍정 키워드 리스트'와 '부정 키워드 리스트'의 Top20단어를 출력
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
invert_index_vectorizer

In [ ]:
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])

In [ ]:
for coef in coef_neg_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])

키워드를 살펴보면:

이용객들이 보통 제주 호텔의 바다뷰 혹은 바다 접근성, 주변 맛집 그리고 인테리어 등에 만족하는 것으로 보입니다.
하지만 숙소의 냄새 그리고 침대, 에어컨 등 시설의 상태가 많이 아쉬워 보이고 개선이 필요해 보임.

# 영화 시나리오 : word cloud & 단어 중요도(TF-IDF)분석

## 1. Libray & data import

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## 사용할 데이터셋

영화 'The Bourne Supermacy'의 시나리오를 활용

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/bourne_scenario.csv")

In [ ]:
df.head()

* Feature Description
 - page_no : 데이터가 위치한 pdf파일의 페이지 수
 - scence-title :씬 제목
 - text : 씬에 해당하는 지문/대본 텍스트 정보
 

In [ ]:
df.shape

In [ ]:
#결측치
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# text변수 확인
df['text'][0]

우리가 필요없는 내용들이 포함되어 있다. 맨 앞에 있는 씬 번호, 공백,특수문자 등 이들을 제거하는 전처리 과정이 필요함
또한, Text mining을 진행할때, 대소문자의 구분이 의미가 없다. 따라서 대문자를 소문자로 변환하는 작업도 진행

- 텍스트 데이터 전처리
  - 정규 표현식 적용

In [ ]:
df['text'][0]

In [ ]:
# 정규 표현식 함수 정의

import re

def apply_regular_expression(text):
    text = text.lower()  # 대문자 -> 소문자 변환
    english = re.compile('[^ a-z]')  # 영어 추출 규칙: 띄어 쓰기를 포함한 알파벳
    result = english.sub('', text)  # 위에 설정한 "english"규칙을 "text"에 적용(.sub)시킴
    result = re.sub(' +', ' ', result) # 2개 이상의 공백을(' +') 하나의 공백(' ')으로 바꿈
    return result

In [ ]:
apply_regular_expression(df['text'][0])

소문자만 존재하고, 공백과 특수문자가 모두 제거

In [ ]:
#정규 표현식 적용
df['processed_text'] = df['text'].apply(lambda x:apply_regular_expression(x))
df.head()

## Word Count
- 말뭉치(코퍼스) 생성

In [ ]:
corpus =df['processed_text'].tolist()
corpus

- BOW(bag of words) 벡터 생성

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# filter stop words
vect = CountVectorizer(tokenizer=None, stop_words='english', analyzer='word').fit(corpus)
# tokenize: 문장을 단어로 나누는 기준; stop_words: 불용어 설정

bow_vect = vect.fit_transform(corpus) # BoW 벡터 생성
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [ ]:
#등장한 단어 list
word_list

In [ ]:
#각 단어의 씬별 등장 횟수

bow_vect.toarray()

In [ ]:
bow_vect.shape

In [ ]:
#각 단어의 총 등장 횟수(모든 씬에서의 등장 횟수의 합)

count_list #BoW array의 각 Column에 대해서 모든 row의 합을 구하기

In [ ]:
#'단어' - '총 등장 횟수' matching

word_count_dict = dict(zip(word_list, count_list))
word_count_dict

In [ ]:
import operator


sorted(word_count_dict.items(), key = operator.itemgetter(1), reverse = True)[:5]

### 단어 분포 탐색

In [ ]:
plt.hist(list(word_count_dict.values()),bins = 150)
plt.show()

대부분의 단어가 0번~50번 사이에 등장했고, 일부 소수의 단어들이 100번 이상 등장한 것을 확인

# 텍스트 마이닝

## 단어별 빈도 분석
 - 상위 빈도수 단어 출력

In [ ]:
# word_count_dict중 상위 25 tags 확인해보기

ranked_tags = Counter(word_count_dict).most_common(25)
ranked_tags

 - word cloud 시각화
 

In [ ]:
!pip install pytagcloud pygame simplejson

In [ ]:
from collections import Counter

import random
import pytagcloud
import webbrowser

In [ ]:
# Top 40 단어로 word cloud 생성하기
taglist = pytagcloud.make_tags(sorted(word_count_dict.items(), key = operator.itemgetter(1), reverse=True)[:40], maxsize=60)  # 빈도수(itemgetter(1)) 내림차순(reverse=True)으로 정렬, maxsize: 글자 크기
# taglist = pytagcloud.make_tages(Counter(word_count_dict).most_common(40), maxsize=60)

pytagcloud.create_tag_image(taglist, 'movie_wordcloud.jpg', rectangular=False)
from IPython.display import Image
Image(filename='movie_wordcloud.jpg')

### 장면별 중요 단어 시각화(TF-IDF)
 - BOW에 대해서 TF-IDF변환 진행
 

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

print(tf_idf_vect.shape)  # 320*2850 vector: 320 scenes, 2850 sentences

In [ ]:
# 첫번째 행 출력 (0이 아닌것 만) -- 즉 첫 씬에서 모든 단어의 TF-IDF 값
print(tf_idf_vect[0])

In [ ]:
# (0을 포함한) 실제 vector의 모습 출력해보기
print(tf_idf_vect[0].toarray().shape)
print(tf_idf_vect[0].toarray())

 - “벡터” - “단어” mapping


길이가 2850인 단어 벡터의 각 위치가 어떤 단어를 상징하는지를 알아내기 위해 단어 벡터에 대해서 “단어” - “index No.” Mapping 을 진행합니다.

In [ ]:
vect.vocabulary_

In [ ]:
# Mapping: 단어 <-> 벡터안의 index no. 
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}  # value : key
print(str(invert_index_vectorizer)[:100]+'...')

- 중요 단어 추출 - Top3 TF-IDF



In [ ]:
np.argsort(tf_idf_vect[0].toarray())[0][-3:]

In [ ]:
#TF-IDF 적용

np.argsort(tf_idf_vect.toarray())[:, -3:]

In [ ]:
top_3_words = np.argsort(tf_idf_vect.toarray())[:, -3:]
df['important_word_index'] = pd.Series(top_3_words.tolist())
df.head()

하지만 지금 중요한 단어의 index만 표시 되고, 과연 어떤 단어인지를 모릅니다. 그래서 우리는 방금 추출한 “벡터”-“단어” Mapping 결과를 이용해 index에 해당하는 단어들을 추출하여 데이터셋에 저장하겠습니다.

In [ ]:
# index -> word 변환함수 만들기

def convert_to_word(x):
    word_list = []
    for index in x:
        word_list.append(invert_index_vectorizer[index])
    return word_list

In [ ]:
df['important_words'] = df['important_word_index'].apply(lambda x: convert_to_word(x))
df.head()